In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from random import seed, randint
from sklearn.metrics import f1_score
from scipy.stats import randint

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
import random
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
import seaborn as sns
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from random import seed, randint
from sklearn.metrics import f1_score
from scipy.stats import randint
from scipy.stats import uniform
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import StratifiedKFold 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import Imputer 
from sklearn.decomposition import PCA 
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, confusion_matrix, recall_score

In [3]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from IPython.display import display # Allows the use of display() for DataFrames

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
#train_df = train_df_copy.copy(deep = True)
# Two optional arguments that you can add to .read_csv() 
    # na_values: Converts a given string to np.nan, defaults to None
        #Add the argument na_values=['NAN'] to pd.read_csv().
    # parse_dates: Reads the data in a list of given columns as dtype datetime64, defaults to False
        #Add parse_dates=['Last Update'] to pd.read_csv()
Id = test_df['PassengerId']
data_cleaner = [train_df, test_df]


#remove Y variable
y = train_df["Survived"]
train_df = train_df.drop(["Survived"], axis = 1)

In [4]:
print('Train columns with null values:\n', train_df.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', test_df.isnull().sum())
print("-"*10)


('Train columns with null values:\n', PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64)
----------
('Test/Validation columns with null values:\n', PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64)
----------


In [5]:
###COMPLETING: complete or delete missing values in train and test/validation dataset
drop_column = ['PassengerId','Cabin', 'Ticket']
for dataset in data_cleaner:    
    #complete missing age with median
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)

    #complete embarked with mode
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)

    #complete missing fare with median
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
    #delete the cabin feature/column and others previously stated to exclude in train dataset
    dataset.drop(drop_column, axis=1, inplace = True)
    

In [6]:
print(train_df.isnull().sum())
print("-"*10)
print(test_df.isnull().sum())

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
----------
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


In [7]:
Title_Dictionary = {
"Capt": "Officer",
"Col": "Officer",
"Major": "Officer",
"Jonkheer": "Royalty",
"Don": "Royalty",
"Sir" : "Royalty",
"Dr": "Officer",
"Rev": "Officer",
"the Countess": "Royalty",
"Dona": "Royalty",
"Mme": "Mrs",
"Mlle": "Miss",
"Ms": "Miss",
"Mr" : "Mr",
"Mrs" : "Mrs",
"Miss" : "Miss",
"Master" : "Master",
"Lady" : "Royalty"
}    

In [8]:
###CREATE: Feature Engineering for train and test/validation dataset
for dataset in data_cleaner:    
    #Discrete variables
    dataset['FamilySize'] = dataset ['SibSp'] + dataset['Parch'] + 1

    dataset['IsAlone'] = 1 #initialize to yes/1 is alone
    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0 # now update to no/0 if family size is greater than 1

    #quick and dirty code split title from name: http://www.pythonforbeginners.com/dictionary/python-split
    dataset['Title'] = dataset['Name'].str.extract('.*, (.*?)\. .*', expand = False)
    dataset['Title'] = dataset['Title'].apply(Title_Dictionary.get)


    #Continuous variable bins; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
    #Fare Bins/Buckets using qcut or frequency bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)

    #Age Bins/Buckets using cut or value bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
    dataset['AgeBin'] = pd.cut(dataset['Age'].astype(int), 5)

#preview data again
train_df.info()
test_df.info()
train_df.sample(10)

C:\Users\203014767\.conda\envs\DAND\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Name          891 non-null object
Sex           891 non-null object
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Embarked      891 non-null object
FamilySize    891 non-null int64
IsAlone       891 non-null int64
Title         891 non-null object
FareBin       891 non-null category
AgeBin        891 non-null category
dtypes: category(2), float64(2), int64(6), object(4)
memory usage: 85.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 13 columns):
Pclass        418 non-null int64
Name          418 non-null object
Sex           418 non-null object
Age           418 non-null float64
SibSp         418 non-null int64
Parch         418 non-null int64
Fare          418 non-null float64
E

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Title,FareBin,AgeBin
234,0,2,"Leyson, Mr. Robert William Norman",male,24.0,0,0,10.5000,S,1,1,Mr,"(7.91, 14.454]","(16.0, 32.0]"
60,0,3,"Sirayanian, Mr. Orsen",male,22.0,0,0,7.2292,C,1,1,Mr,"(-0.001, 7.91]","(16.0, 32.0]"
862,1,1,"Swift, Mrs. Frederick Joel (Margaret Welles Ba...",female,48.0,0,0,25.9292,S,1,1,Mrs,"(14.454, 31.0]","(32.0, 48.0]"
464,0,3,"Maisner, Mr. Simon",male,28.0,0,0,8.0500,S,1,1,Mr,"(7.91, 14.454]","(16.0, 32.0]"
781,1,1,"Dick, Mrs. Albert Adrian (Vera Gillespie)",female,17.0,1,0,57.0000,S,2,0,Mrs,"(31.0, 512.329]","(16.0, 32.0]"
865,1,2,"Bystrom, Mrs. (Karolina)",female,42.0,0,0,13.0000,S,1,1,Mrs,"(7.91, 14.454]","(32.0, 48.0]"
151,1,1,"Pears, Mrs. Thomas (Edith Wearne)",female,22.0,1,0,66.6000,S,2,0,Mrs,"(31.0, 512.329]","(16.0, 32.0]"
458,1,2,"Toomey, Miss. Ellen",female,50.0,0,0,10.5000,S,1,1,Miss,"(7.91, 14.454]","(48.0, 64.0]"
12,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,8.0500,S,1,1,Mr,"(7.91, 14.454]","(16.0, 32.0]"
112,0,3,"Barton, Mr. David John",male,22.0,0,0,8.0500,S,1,1,Mr,"(7.91, 14.454]","(16.0, 32.0]"


In [9]:
cat = ['Sex', 'Embarked', 'Title', 'FamilySize', 'FareBin', 'AgeBin']

In [10]:
cat_df_train_encoded = pd.get_dummies(train_df[cat])
#print(features)
#print(features.shape)
                     
#Print the number of features after one-hot encoding
encoded = list(cat_df_train_encoded.columns)
print "{} total features after one-hot encoding.".format(len(encoded))

21 total features after one-hot encoding.


In [11]:
cat_df_test_encoded = pd.get_dummies(test_df[cat])
#print(features)
#print(features.shape)
                     
#Print the number of features after one-hot encoding
encoded = list(cat_df_test_encoded.columns)
print "{} total features after one-hot encoding.".format(len(encoded))

21 total features after one-hot encoding.


In [12]:
cat_df_train_encoded = pd.DataFrame(cat_df_train_encoded)
cat_df_test_encoded = pd.DataFrame(cat_df_test_encoded)

In [13]:
frames_train = [train_df, cat_df_train_encoded]
train_df = pd.concat(frames_train, axis = 1)


In [14]:
frames_test = [test_df, cat_df_train_encoded]
test_df = pd.concat(frames_test, axis = 1)


In [15]:
###Drop categorical variables
drop_column = ['Name','Sex','Embarked','Fare','SibSp','Parch', 'FareBin','Title','AgeBin','FareBin', 'Age']
#for dataset in data_cleaner:    
#    dataset.drop(drop_column, axis=1, inplace = True)

In [16]:
test_df.drop(drop_column, axis=1, inplace = True)

In [17]:
train_df.drop(drop_column, axis=1, inplace = True)

In [26]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 24 columns):
Pclass                     418 non-null float64
FamilySize                 418 non-null float64
IsAlone                    418 non-null float64
FamilySize                 891 non-null int64
Sex_female                 891 non-null uint8
Sex_male                   891 non-null uint8
Embarked_C                 891 non-null uint8
Embarked_Q                 891 non-null uint8
Embarked_S                 891 non-null uint8
Title_Master               891 non-null uint8
Title_Miss                 891 non-null uint8
Title_Mr                   891 non-null uint8
Title_Mrs                  891 non-null uint8
Title_Officer              891 non-null uint8
Title_Royalty              891 non-null uint8
FareBin_(-0.001, 7.91]     891 non-null uint8
FareBin_(7.91, 14.454]     891 non-null uint8
FareBin_(14.454, 31.0]     891 non-null uint8
FareBin_(31.0, 512.329]    891 non-null uint8
AgeBin_(-0.08, 

In [19]:
#remove Y variable 
y = train_df["Survived"] 

In [20]:
train_df = train_df.drop(["Survived"], axis = 1)

### Train and Test Feature Creation

In [21]:
sss = StratifiedKFold(n_splits = 4, random_state=42)
kfold = KFold(n_splits=10, random_state=seed)
num_trees = 100
num_folds = 5
scoring = 'roc_auc'
seed = 7
scaler = MinMaxScaler(feature_range=(0, 1))

In [22]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split = \
    train_test_split(train_df, y, test_size=0.2, random_state=42)

C:\Users\203014767\.conda\envs\DAND\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [23]:
# base case
# Spot-Check Algorithms
LR_clf = LogisticRegression()
LDA_clf = LinearDiscriminantAnalysis()
KNN_clf = KNeighborsClassifier()
RF_clf = RandomForestClassifier(random_state = 7)
NB_clf = GaussianNB()
SVC_clf = SVC()
ADA_clf = AdaBoostClassifier(random_state = 7)
GB_clf = GradientBoostingClassifier()

models = []
models.append(('LR', LR_clf))
models.append(('LDA', LDA_clf))
models.append(('KNN', KNN_clf))
models.append(('RF',RF_clf))
models.append(('NB', NB_clf))
models.append(('SVM', SVC_clf))
models.append(('ADA', ADA_clf))
models.append(('GB', GB_clf))

results = []
names = []
for name, model in models:
    cv_results = cross_val_score(model, X_train, y_train, cv=sss, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.857335 (0.035337)


C:\Users\203014767\.conda\envs\DAND\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LDA: 0.853267 (0.042621)
KNN: 0.830510 (0.041837)
RF: 0.823467 (0.041675)
NB: 0.835888 (0.038630)
SVM: 0.830073 (0.046364)
ADA: 0.856797 (0.038114)
GB: 0.854948 (0.026402)


In [24]:
#Scaled

pipelines = []
pipelines.append(('LR', Pipeline([('Scaler', scaler),('LR',LR_clf)])))
pipelines.append(('LDA', Pipeline([('Scaler', scaler),('LDA',LDA_clf)])))
pipelines.append(('KNN', Pipeline([('Scaler', scaler),('KNN',KNN_clf)])))
pipelines.append(('RF', Pipeline([('Scaler', scaler),('RF',RF_clf)])))
pipelines.append(('NB', Pipeline([('Scaler', scaler),('NB',NB_clf)])))
pipelines.append(('SVM', Pipeline([('Scaler', scaler),('SVM', SVC_clf)])))
pipelines.append(('ADA', Pipeline([('Scaler', scaler),('ADA', ADA_clf)])))
pipelines.append(('GB', Pipeline([('Scaler', scaler),('GB', GB_clf)])))
results_sss = []
names_sss = []
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=sss, scoring=scoring)
    results_sss.append(cv_results)
    names_sss.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.856696 (0.037969)
LDA: 0.853267 (0.042621)
KNN: 0.808760 (0.032408)
RF: 0.823467 (0.041675)
NB: 0.835888 (0.038630)
SVM: 0.830510 (0.060216)
ADA: 0.856797 (0.038114)
GB: 0.854881 (0.026383)


In [25]:
#Tuned

pipelines = []
pipelines.append(('LR', Pipeline([('Scaler', scaler),('LR',LR_clf_tuned_scaled)])))
pipelines.append(('LDA', Pipeline([('Scaler', scaler),('LDA',LDA_clf_tuned_scaled)])))
pipelines.append(('KNN', Pipeline([('Scaler', scaler),('KNN',KNN_clf_tuned_scaled)])))
pipelines.append(('RF', Pipeline([('Scaler', scaler),('RF',RF_clf_tuned_scaled)])))
pipelines.append(('NB', Pipeline([('Scaler', scaler),('NB',NB_clf)])))
pipelines.append(('SVM', Pipeline([('Scaler', scaler),('SVM', SVC_clf_tuned_scaled)])))
pipelines.append(('ADA', Pipeline([('Scaler', scaler),('ADA', ADA_clf_tuned_scaled)])))
pipelines.append(('GB', Pipeline([('Scaler', scaler),('GB', GB_clf_tuned_scaled)])))
pipelines.append(('Voting', Pipeline([('Scaler', scaler),('Voting', voting_ensemble)])))
results_scaled_tune = []
names_scaled_tune = []
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=sss, scoring=scoring)
    results_scaled_tune.append(cv_results)
    names_scaled_tune.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

NameError: name 'LR_clf_tuned_scaled' is not defined

In [ ]:
#Compare Algorithms
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

# Compare Algorithms Scaled
fig_sss = plt.figure()
fig_sss.suptitle('Scaled Algorithm Comparison')
ax = fig_sss.add_subplot(111)
plt.boxplot(results_sss)
ax.set_xticklabels(names_sss)
plt.show()

# Compare Algorithms
fig_tune_scaled = plt.figure()
fig_tune_scaled.suptitle('Tuning Algorithm Comparison')
ax = fig_tune_scaled.add_subplot(111)
plt.boxplot(results_scaled_tune, showmeans = True, meanline=False)
ax.set_xticklabels(names_scaled_tune)
plt.show()


### Data Preparation

In addition to performing transformations on features that are highly skewed, it is often good practice to perform some type of scaling on numerical features. Applying a scaling to the data does not change the shape of each feature's distribution (such as 'capital-gain' or 'capital-loss' above); however, normalization ensures that each feature is treated equally when applying supervised learners. Note that once scaling is applied, observing the data in its raw form will no longer have the same original meaning, as exampled below.

In [ ]:
# scale data between 0 and 1
scaler.fit(X_train)
X_train_scaled =scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Tuning with All Features

In [ ]:
# Tune scaled KNN
pipeline =Pipeline([
        ('Scaler', scaler),
        ('clf',KNN_clf) 
         ])
param_grid = dict(clf__n_neighbors=range(1,12))
grid = GridSearchCV(pipeline, param_grid=param_grid, scoring=scoring, cv=sss)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
KNN_clf_tuned_scaled = grid.best_estimator_

In [ ]:
# Tune scaled SVM
c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']
prob =[True, False]
pipeline =Pipeline([
        ('Scaler', scaler),
        ('clf',SVC_clf) 
         ])
param_grid = dict(clf__C=c_values, clf__kernel=kernel_values, clf__probability = prob)
grid = GridSearchCV(pipeline, param_grid=param_grid, scoring=scoring, cv=sss)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
SVC_clf_tuned_scaled = grid.best_estimator_


In [ ]:
# Tune LR
c_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
penalty_values = ['l1', 'l2']
pipeline =Pipeline([
        ('Scaler', scaler),
        ('clf',LR_clf) 
         ])
param_grid = dict(clf__C=c_values, clf__penalty=penalty_values)
grid = GridSearchCV(pipeline, param_grid=param_grid, scoring=scoring, cv=sss)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
LR_clf_tuned_scaled = grid.best_estimator_

In [ ]:
# Tune LDA
solver_values = ['svd', 'lsqr']
pipeline =Pipeline([
        ('Scaler', scaler),
        ('clf',LDA_clf) 
         ])
param_grid = dict(clf__solver=solver_values)
grid = GridSearchCV(pipeline, param_grid=param_grid, scoring=scoring, cv=sss)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
LDA_clf_tuned_scaled = grid.best_estimator_


In [ ]:
LDA_clf_tuned_scaled

In [ ]:
RF_clf.get_params().keys()

In [ ]:
#https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/
#Tune RF
pipeline =Pipeline([
        ('Scaler', scaler),
        ('clf',RF_clf) 
         ])

param_grid = {
    'clf__max_depth' : randint(low=1, high=5),
    'clf__max_features':uniform(),
    'clf__min_samples_split': randint(low=2, high=5),
    'clf__n_estimators' : randint(low=100, high=200) }

rsearch = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=25, cv = sss, 
                             random_state=7, scoring = scoring)
rsearch_result = rsearch.fit(X_train, y_train)
#print(rsearch.best_score_)
#print(rsearch.best_estimator_)
#print(rsearch.best_params_)

print("Best: %f using %s" % (rsearch_result.best_score_, rsearch_result.best_params_))
RF_clf_tuned_scaled = rsearch_result.best_estimator_

Best: 0.860946 using {'clf__max_features': 0.90912837488673126, 'clf__max_depth': 3, 'clf__n_estimators': 187, 'clf__min_samples_split': 3}

In [ ]:
# Tune ADA - First tune number of Trees
pipeline =Pipeline([
        ('Scaler', scaler),
        ('clf',ADA_clf) 
         ])
param_grid = {#'learning_rate' : uniform(),            
'clf__n_estimators' : range(1, 100, 5) }

grid = GridSearchCV(pipeline, param_grid=param_grid, scoring=scoring, cv=sss)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
ADA_clf_tuned_scaled = grid.best_estimator_
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))
#ADA_clf_tuned = grid_search.best_estimator_
#print(rsearch.best_score_)
#print(rsearch.best_estimator_)
#print(rsearch.best_params_)

#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#GB_clf_tuned = grid_result.best_estimator_

In [ ]:
GB_clf.get_params().keys()

In [ ]:
# Tune GB - First tune number of Trees
pipeline =Pipeline([
        ('Scaler', scaler),
        ('clf',GB_clf) 
         ])
param_grid = {#'learning_rate' : uniform(), 
'clf__max_depth' : [2, 4, 6, 8, 10],
#'subsample': uniform(),            
'clf__n_estimators' : range(100, 1000, 100) }

grid = GridSearchCV(pipeline, param_grid=param_grid, scoring=scoring, cv=sss)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
GB_clf_tuned_scaled = grid.best_estimator_
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
GB_clf.get_params().keys()

In [ ]:
pipeline =Pipeline([
        ('Scaler', scaler),
        ('clf',GB_clf) 
         ])

param_grid = {'clf__learning_rate' : uniform(0.02,0.1),
'clf__objective': ['binary:logistic'],
'clf__gamma': [0],
'clf__min_child_weight': uniform(),              
'clf__max_depth': randint(low=6, high=12),
'clf__subsample': [1.0],#row sampling              
'clf__colsample_bytree':[0.3, 0.4, 0.5],
'clf__n_estimators' : [100] }


rsearch = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=50, cv = sss, 
                             random_state=7, scoring = scoring)

rsearch_result = rsearch.fit(X_train, y_train)
print("Best: %f using %s" % (rsearch_result.best_score_, rsearch_result.best_params_))
GB_clf_tuned_scaled = rsearch_result.best_estimator_

Best: 0.876123 using {'clf__gamma': 2.7159522025956386, 'clf__n_estimators': 103, 'clf__max_depth': 9, 'clf__learning_rate': 0.27721958269429525, 'clf__objective': 'binary:logistic', 'clf__subsample': 0.69359919451504393}

Best: 0.868762 using {'clf__gamma': 0, 'clf__n_estimators': 100, 'clf__max_depth': 10, 'clf__learning_rate': 0.023552880483036338, 'clf__min_child_weight': 0.38679069137477007, 'clf__colsample_bytree': 0.4, 'clf__objective': 'binary:logistic', 'clf__subsample': 1.0}

In [ ]:
# Voting
# create the sub models
estimators = []
model1 = RF_clf_tuned_scaled
estimators.append(('RF', model1))
#model2 = LR_clf_tuned_scaled
#estimators.append(('LR', model2))
model3 = GB_clf_tuned_scaled
estimators.append(('GB', model3))
#model4 = ADA_clf_tuned_scaled
#estimators.append(('ADA', model3))
# create the ensemble model
voting_ensemble = VotingClassifier(estimators, voting = 'soft' )
results = cross_val_score(voting_ensemble, X_train_scaled, y_train, cv = sss, scoring=scoring)
print(results.mean())

In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
title = "Learning Curves (LR Tuned)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
plot_learning_curve(LR_clf_tuned_scaled, title, X_train_scaled, y_train, cv=sss, n_jobs=1)


title = "Learning Curves (LDA Tuned)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
plot_learning_curve(LDA_clf_tuned_scaled, title, X_train_scaled, y_train, cv=sss, n_jobs=1)


title = "Learning Curves (KNN Tuned)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
plot_learning_curve(KNN_clf_tuned_scaled, title, X_train_scaled, y_train, cv=sss, n_jobs=1)


title = "Learning Curves (NB)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
plot_learning_curve(NB_clf, title, X_train_scaled, y_train, cv=sss, n_jobs=1)

title = "Learning Curves (SVC Tuned)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
plot_learning_curve(SVC_clf_tuned_scaled, title, X_train_scaled, y_train, cv=sss, n_jobs=1)

title = "Learning Curves (ADA Tuned)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
plot_learning_curve(ADA_clf, title, X_train_scaled, y_train, cv=sss, n_jobs=1)



In [ ]:
title = "Learning Curves (GB UnTuned)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
plot_learning_curve(GB_clf, title, X_train_scaled, y_train, cv=sss, n_jobs=1)


title = "Learning Curves (RF Tuned)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
plot_learning_curve(RF_clf_tuned_scaled, title, X_train_scaled, y_train, cv=sss, n_jobs=1)

title = "Learning Curves (GB Tuned)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
plot_learning_curve(GB_clf_tuned_scaled, title, X_train_scaled, y_train, cv=sss, n_jobs=1)

title = "Learning Curves (Voting Tuned)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
plot_learning_curve(voting_ensemble, title, X_train_scaled, y_train, cv=sss, n_jobs=1)

In [ ]:
RF_clf_tuned_scaled.fit(X_train_scaled, y_train)
RF_test_pred = RF_clf_tuned_scaled.predict(X_test_scaled)
RF_prec_test = precision_score(RF_test_pred, y_test)
RF_recall_test = recall_score(RF_test_pred, y_test)
RF_f1_test = f1_score(RF_test_pred, y_test)
RF_ROC_AUC_test = roc_auc_score(RF_test_pred, y_test)

print("Precision Test: {}".format(RF_prec_test))
print ("Recall Test: {}".format(RF_recall_test))
print ("F1 Test: {}".format(RF_f1_test))
print ("ROC AUC Test: {}".format(RF_ROC_AUC_test))

In [ ]:
LR_clf_tuned_scaled.fit(X_train_scaled, y_train)
SVC_test_pred = LR_clf_tuned_scaled.predict(X_test_scaled)
SVC_prec_test = precision_score(SVC_test_pred, y_test)
SVC_recall_test = recall_score(SVC_test_pred, y_test)
SVC_f1_test = f1_score(SVC_test_pred, y_test)
SVC_ROC_AUC_test = roc_auc_score(SVC_test_pred, y_test)

print("Precision Test: {}".format(SVC_prec_test))
print ("Recall Test: {}".format(SVC_recall_test))
print ("F1 Test: {}".format(SVC_f1_test))
print ("ROC AUC Test: {}".format(SVC_ROC_AUC_test))

In [ ]:
GB_clf_tuned_scaled.fit(X_train_scaled, y_train)
GB_test_pred = GB_clf_tuned_scaled.predict(X_test_scaled)
GB_prec_test = precision_score(GB_test_pred, y_test)
GB_recall_test = recall_score(GB_test_pred, y_test)
GB_f1_test = f1_score(GB_test_pred, y_test)
GB_ROC_AUC_test = roc_auc_score(GB_test_pred, y_test)

print("GBC Precision Test: {}".format(GB_prec_test))
print ("GBC Recall Test: {}".format(GB_recall_test))
print ("GBC F1 Test: {}".format(GB_f1_test))
print ("GBC ROC AUC Test: {}".format(GB_ROC_AUC_test))

In [ ]:
ADA_clf_tuned_scaled.fit(X_train_scaled, y_train)
ADA_test_pred = ADA_clf_tuned_scaled.predict(X_test_scaled)
ADA_prec_test = precision_score(ADA_test_pred, y_test)
ADA_recall_test = recall_score(ADA_test_pred, y_test)
ADA_f1_test = f1_score(ADA_test_pred, y_test)
ADA_ROC_AUC_test = roc_auc_score(ADA_test_pred, y_test)

print("Precision Test: {}".format(ADA_prec_test))
print ("Recall Test: {}".format(ADA_recall_test))
print ("F1 Test: {}".format(ADA_f1_test))
print ("ROC AUC Test: {}".format(ADA_ROC_AUC_test))

In [ ]:
voting_ensemble

voting_ensemble.fit(X_train_scaled, y_train)
voting_test_pred = voting_ensemble.predict(X_test_scaled)
voting_prec_test = precision_score(voting_test_pred, y_test)
voting_recall_test = recall_score(voting_test_pred, y_test)
voting_f1_test = f1_score(voting_test_pred, y_test)
voting_ROC_AUC_test = roc_auc_score(voting_test_pred, y_test)

print("Precision Test: {}".format(voting_prec_test))
print ("Recall Test: {}".format(voting_recall_test))
print ("F1 Test: {}".format(voting_f1_test))
print ("ROC AUC Test: {}".format(voting_ROC_AUC_test))

### Scale and Transform Test Data

In [ ]:
test_df.info()

In [ ]:
test_scaled  = scaler.transform(test_df)

In [ ]:
y_pred = voting_ensemble.predict(test_scaled)
print(y_pred)

# Generate Submission File 
submission = pd.DataFrame({ 'PassengerId': Id,
                            'Survived': y_pred })
submission.to_csv("submission.csv", index=False)